 # Organic-Order 🍇🍅🥕

<div class="alert alert-info">
    <h3>Please read the following important information before starting with the programming exercise: </h3>
    <p>In order to avoid problems with the relative file path we recommend to clone the exercise repository into the <b>homework</b> folder and you will <b>get all provided files</b> in the <b>homework/ai23logic-"your_TUM_ID"</b> folder.</p> 
    <p><b>Only modify</b> the rules.py and <b>Do not</b> change the signatures of functions we provided you!</p>
    <p>Changes to other files will be ignored by the tests on Artemis</p>
    <p>If we are not able to run your submitted files in an environment with the packages provided by the requirements.txt of the <b><a href="https://github.com/aimaTUM/aima-python">AIMA repository</a></b>, you will fail the programming exercise.</p> 
</div>

# Initialization

In [1]:
import os
import sys

# Importing AIMA tools
# If you have not copied the contents of the template directly into the AIMA folder, you can uncomment the next line.
sys.path.insert(0, os.path.realpath("../../aima-python"))  # Modify the ./aima to the relative path of your aima folder.
# For the docker installation, you can use
sys.path.insert(0, os.path.realpath("/work"))
# Do not change the rest of this block

import numpy as np
from field_var import field_var
from ask_solution import ask_solution
from utils import expr
from kb import DpllPropKB
from visualize import draw, draw_solution
import configurations as confs
from rules import OnePlantPerCellConstraint, OneCellPerPlantConstraint, NextToConstraint, NotNextToConstraint, ToTheRightConstraint, ToTheLeftConstraint, IfThenNotNextToConstraint, EitherOrConstraint, IfThenConstraint, InitialLayoutConstraint
from generate_knowledge import generate_knowledge

KB = DpllPropKB

# Introduction

### Problem Description

[Organic Order](https://www.lumosity.com/en/brain-games/organic-order/) is a brain-training game developed by Lumosity, it focuses on logical reasoning by arranging fruits or vegetables in a specific order, following explicit stated rules and implicit ones inferred through observation and deduction.

The initial configuration is given as a list of strings. This list has length x, corresponding to the x organic elements that may be involved in this scenario. The elements of the list specify the initial positions of certain elements. For example, `['Carrot','*','Grape','*','*']` represents the following initial ordering: TODO (image)

Additionally, a list of logical statements are given; from these rules, implicit rules have to be deduced. The goal is to assign a position to each plant into the initial layout while following all given rules. This is achieved by filling a knowledge base with logical sentences.

### Passing the exercise
You will have passed the exercise if you can successfully solve all **4 general** and **7 rule-specific scenarios** that are given to you together with this template. Additionally, your code also has to solve **2 hidden scenarios**, that are only available on Artemis. 

_Note: you cannot solve this exercise by hardcoding the solutions for the scenarios given in this template. In order to also solve the scenarios hidden on Artemis, you need to implement the knowledge base with logical sentences in a correct way._

If your implemented function computes the knowledge base such that every scenario is solved correctly (including the hidden ones), you successfully completed this programming exercise. Your code has to compute a valid solution for **each** of the scenarios **within 5 min** on our machine. If your code takes longer to compute a solution, you will fail this submission. Don't worry about the computation time too much as usually the algorithm produces a solution **within two minutes** for our specific exercise. Your submission will be evaluated after the deadline, but until then you can update your solution as many times as you like. The last submitted solution will be graded.


# Visualization

For this exercise, several initial layouts and sets of rules are given. Each layout is represented as a list. The i-th entry of this array corresponds to the (i+1)-th field on the map, so **init_layout[0]** is the **1st** and leftmost cell. Each layout can have different sizes ranging from 4 to 6.

**The layout is an array of the following elements, presenting either an empty cell or a type of plant.**

- **'*'** means that the cell is empty
- **'C'** denotes Carrot
- **'T'** denotes Tomato
- **'B'** denotes Berries
- **'G'** denotes Grape
- **'P'** denotes Peas
- **'Cn'** denotes Corn

In order to visualize the layout you can use the following function:

In [2]:
# You can choose an initial configuration by setting the variable conf_index (possible values: 1 to 11)
conf_index = 1
init_layout = getattr(confs, "layout" + str(conf_index))
rule_list = getattr(confs, "rules" + str(conf_index))
to_be_assigned = getattr(confs, "to_be_assigned" + str(conf_index))
exact_solution = getattr(confs, "solution" + str(conf_index))

# init_layout is a list consisting the placement of given plants: 
print(f'The initial layout :\n{draw(init_layout)}')

The initial layout :
* | * | Peas | * | *


In [ ]:
# You can access cell elements x with init_layout[x-1]
x = 2
print(f'The element at position {x} is', init_layout[x - 1])

# Knowledge Base generation

## Atomic sentences

Your task is to implement the knowledge base that covers all implicit logical constraints inferred from the rules and initial layout. To represent the placement of an organic element, use one of the five predicates as shown below:

In [ ]:
# Formalization of the sentence 'Peas is in cell 1':
field_var(fruit="Peas", loc=1)

In [ ]:
# Formalization of the sentence 'Berries is in cell 5':
field_var(fruit="Berries", loc=5)

In [ ]:
# Formalization of the sentence 'Grape is in cell 4':':
field_var(fruit="Grape", loc=4)

In [ ]:
# Formalization of the sentence 'Carrot is in cell 3':
field_var(fruit="Carrot", loc=3)

In [ ]:
# Formalization of the sentence 'Tomato is in cell 2':
field_var(fruit="Tomato", loc=2)

In [ ]:
# Formalization of the sentence 'Corn is in cell 1':
field_var(fruit="Corn", loc=1)

## Telling and asking about the knowledge base

With the atomic sentences introduced above, you should construct correct logical sentences that can be used to infer the ordering of the plants. Below you will find an example detailing the notation you can use to construct such sentences, and how to add them to the knowledge base. _**Note:**_ The next logical sentences are **only** meant to illustrate the notation, and **do not** necessarily reflect real scenarios or solutions.


In [ ]:
# Notation you can use:
# &     Logical AND
# |     Logical OR
# ~     Negation
# ==>   Implication
# <==   Reverse Implication
# <=>   Equivalence
# (...) Parentheses

# Example of a simple knowledge base construction
# Create an empty knowledge base
kb = KB()

# Formalize the sentences
# 'Grape is in cell 2'
# 'Carrot is in cell 4'
# 'Tomato is not in cell 1'
sentence1 = field_var(fruit="Grape", loc=2)
sentence2 = field_var(fruit="Carrot", loc=4)
sentence3 = f'~{field_var(fruit="Tomato", loc=1)}'
# Add these sentences to the KB 
kb.tell(expr(sentence1))
kb.tell(expr(sentence2))
kb.tell(expr(sentence3))

# Formalize the sentence
# 'if grape is in cell 2 and Tomato is not in cell 1, then Berries is in cell 1
sentence4 = f'{field_var(fruit="Grape", loc=2)}&~{field_var(fruit="Tomato", loc=1)}==>{field_var(fruit="Berries", loc=1)}'
print(sentence4)
# Add this sentence to the KB
kb.tell(expr(sentence4))

# We can now 'ask' the knowledge base whether Berries is in cell 1
kb.ask(expr(field_var(fruit="Berries", loc=1)))

## Important note
If something is _undefined_ in your knowledge base, or if the inference fails, `kb.ask` will always return `False`.
Similarly, if you have _contradictory_ knowledge in your knowledge base, `kb.ask` will always return `True`.
Therefore, make sure that you define your knowledge base properly.

The following example shows how an incorrectly defined knowledge base can lead to problems:

In [ ]:
# Create a new, empty knowledge base:
kb = KB()

# Add the sentence 'If Grape is in cell 1 then Carrot is in cell 2'
sentence = f'{field_var(fruit="Grape", loc=1)}==>{field_var(fruit="Carrot", loc=2)}'
kb.tell(expr(sentence))

# If you do not tell the knowledge base that grape is in cell 1, both of the following will return False:
print(kb.ask(expr(field_var(fruit="Grape", loc=1))))
print(kb.ask(expr(f'~{field_var(fruit="Carrot", loc=2)}')))

# Undefined sentences will also always yield false
print(kb.ask(expr(f'~{field_var(fruit="Tomato", loc=1)}')))

In [ ]:
# If you add contradictory knowledge into the kb ...
# Berries is in cell 1 and Berries is not in cell 1
sentence2 = field_var(fruit="Berries", loc=1)
sentence3 = f'~{field_var(fruit="Berries", loc=1)}'
kb.tell(expr(sentence2))
kb.tell(expr(sentence3))

# ... both of the following will return True:
print(kb.ask(expr(field_var(fruit="Berries", loc=1))))
print(kb.ask(expr(f'~{field_var(fruit="Berries", loc=1)}')))

## Generating your knowledge base

For this exercise, you are required to generate the sentences that solve Organic Order. These sentences will then be added to the knowledge base. Here is what this could look like. This is only an example, and you need to implement your own knowledge base later on!

In addition to the initial layout, a set of rules are also provided and must be used for inference. The possible rules and their corresponding classes are:

- **NextToConstraint**: A must be next to B
- **NotNextToConstraint**: A cannot be next to B
- **ToTheRightConstraint**: A must be next to and right of B
- **ToTheLeftConstraint**: A must be next to and left of B
- **EitherOrConstraint**: A or B must be in position x
- **IfThenConstraint**: If A is in position x, then B must be in position y
- **IfThenNotNextToConstraint**: If A is left to B, then A cannot be next to C

Where A, B, C are fruits/vegetables and x, y are cells/positions in `range(1, len(layout) + 1)`

For each of these rules, a dataclass is defined in **rules.py**, and each class has a function `formalize()` that you should implement.
The output of the function should be a list of expressions that must be added into the knowledge base.
**Do not change the signature of** `formalize()` **or add new members variables to the class.**
Defining helper functions inside the classes is fine though.

The initialization of a rule object takes 2 to 4 arguments depending on its definition, in the following code snippet you will see some examples.

In [ ]:
# Example usage:
# Rule1: (Tomato) must be next to (Carrot)
rule1 = NextToConstraint("Tomato", "Carrot")

# Rule2: (Grape) must be to the right of (Carrot)
rule2 = ToTheRightConstraint("Grape", "Carrot")

# Rule3: If (Peas) is left to (Grape), then (Peas) cannot be next to (Berries)
rule3 = IfThenNotNextToConstraint("Peas", "Grape", "Berries")

# Rule4: (Corn) or (Grape) must be in position 2
rule4 = EitherOrConstraint("Corn", "Grape", 2)

# Rule5: If (Grape) is in position 3, then (Tomato) must be in position 1
rule5 = IfThenConstraint("Grape", 3, "Tomato", 1)

The `formalize()` methods for the above mentioned rules take an argument called `layout_size`, this has to be given since doing inference on different layout sizes with the same rule will result in different knowledge bases.

In [ ]:
# call formalize() to do inference based on the rule
print(rule1.formalize(layout_size=5))

## Your Task

Implement the ``formalize()`` methods in **rules.py**. They should set up the initial knowledge as well as logic of each rule.

Please do not import any additional modules or packages, otherwise your solution will be marked as failed. This exercise is easily solvable without any additional packages.

Other than the rules mentioned above, there are two more implicit rules that ensure the following boundary conditions:
- **OnePlantPerCellConstraint**: At most one plant can occupy one cell
- **OneCellPerPlantConstraint**: Each plant type must occur exactly once
Both of these rules store all fruits that can occur in the puzzle in the list `fruit_types`.

Moreover, you need to respect the initial fruit layout. This is handled via the rule
- **InitialLayoutConstraint**: The initial layout must be respected
You have to ensure that the already placed fruits stay in their designated spot.

In case you want to create and test further configurations, the parameters of `generate_knowledge()` are:
- **init_layout**: a list representing the initial assignment of the plants, for example: `['*','Carrot','Tomato','*', '*']`
- **rule_list**: list of rules in the form of dataclasses, for example: `[EitherOrConstraint("Peas", "Tomato", 1), NotNextToConstraint("Carrot", "Tomato", 3)]`
- **to_be_assigned**: a list of plants that need to go in the empty cells, for example: `['Peas','Grape','Berries']`

<div class="alert alert-danger">
    <p>Modify only the formalize() methods in rules.py, changes to other files will be ignored by the tests on Artemis. Also, do not change the signature of the formalize() methods or add new member variables to the rule classes. Defining helper functions inside the classes is fine though.</p>
</div>

In [ ]:
# # for debugging
# for sentence in generate_knowledge(init_layout, rule_list, to_be_assigned):
#    print(sentence)

# Running the Algorithm



To test your solution, choose an initial configuration by modifying `conf_index`, the configurations 1 to 4 are general tests while configurations 5 - 11 are specific to each of the rules. Check `configurations.py` for more details.

**Note: The evaluation runtime of your solution depends heavily on the size of your knowledge base, but it should take no more than two minutes for each of your solutions. Otherwise, you can check whether you can remove some of the expressions to speed up the inference process, while still ensuring completeness.**

In [3]:
# Create an empty knowledge base, and fill it with your knowledge base
kb = KB()

print("feed knowledge base with knowledge..")
for str_expr in generate_knowledge(init_layout, rule_list, to_be_assigned):
    kb.tell(expr(str_expr))

# check if the knowledge base is obviously wrong (you can remove this if it is too slow)
print("scan knowledge base for contradictions..")
assert not kb.has_contradicting_knowledge()

print("start inference..")
layout_solution = ask_solution(kb)

print("Your solution is:")
draw_solution(init_layout, layout_solution)
print("The actual solution is:")
draw_solution(init_layout, exact_solution)

feed knowledge base with knowledge..
scan knowledge base for contradictions..
start inference..
Your solution is:
[WARNING] You didn't assign a fruit to position 2.
Tomato | * | Peas | Carrot | Grape
The actual solution is:
Berries | Tomato | Peas | Carrot | Grape
